In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
from selenium.webdriver.chrome.options import Options
import time

In [42]:
Headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
content=requests.get('https://www.bigbasket.com/product/all-categories/',headers=Headers).text
soup=BeautifulSoup(content,'html.parser')
categories=soup.find_all('div',class_='dp_headding')
Category=[]
for category in categories:
  Category.append(category.text)

In [43]:
selected_category = ['Foodgrains, Oil & Masala',
 'Bakery, Cakes & Dairy',
 'Beverages',
 'Snacks & Branded Foods',
 'Beauty & Hygiene',
 'Cleaning & Household',
 'Kitchen, Garden & Pets',
 'Baby Care']

In [65]:
categories = []
for heading, cover in zip(soup.find_all('div',class_='dp_headding'), soup.find_all('div',class_='uiv2-search-category-listing-cover')):
    # if heading.text in selected_category:
    for s in cover:
        subcat1soup=s.find_all('div',class_='uiv2-search-cover')
        for Sub in subcat1soup:
            subCat=Sub.find('span').text
            for subCat2soup in Sub.find_all('li'):
                subCat2 = subCat2soup.text
                categories.append({
                    "category": heading.text,
                    "sub_category_1": subCat,
                    "sub_category_2": subCat2,
                    "link": "https://www.bigbasket.com"+subCat2soup.find('a').get("href")+"#!page=1"
                })

In [66]:
with open("./assets/categories.json", "w") as f:
    json.dump(categories,f, indent=2)

In [7]:
def scrape_webpage(webpage_url:str):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=chrome_options)
    # driver.get("https://www.bigbasket.com/pc/foodgrains-oil-masala/masalas-spices/herbs-seasoning/?nc=cs#!page=1")
    driver.get(webpage_url)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(15)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "img"))
    )
    html = driver.page_source
    driver.quit()
    return BeautifulSoup(html, 'html.parser')


In [8]:
def parse_soup(soup):
    products_soup = soup.find("section", class_="z-10")
    products_list = []
    for product_soup in products_soup.find_all("li"):
        name = product_soup.find("div", class_="break-words h-10 w-full")
        rating = product_soup.find("span", class_="Label-sc-15v1nk5-0 gJxZPQ")
        sku_type = product_soup.find("span", class_="Label-sc-15v1nk5-0 gJxZPQ truncate")
        brand_name = product_soup.find("span", class_="Label-sc-15v1nk5-0 BrandName___StyledLabel2-sc-hssfrl-1 gJxZPQ keQNWn")
        image_src = None
        if img:=product_soup.find("img"):
            image_src = img.get("src")
        if image_src:
            products_list.append({
            "name": getattr(name,"text", None),
            "sku_type": getattr(sku_type,"text", None),
            "rating": getattr(rating,"text", None),
            "image_link": image_src
        })
    print("Total products in Cat: ", len(products_soup.find_all("li")))
    print("Products with image: ", len(products_list))
    return products_list

In [9]:
import os
import traceback

In [10]:
from tqdm import tqdm
import re

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [12]:
def process_category(category):
    try:
        category_name = re.sub(r'[^a-zA-Z0-9]', '', category.get("category"))
        sub_category_1 = re.sub(r'[^a-zA-Z0-9]', '', category.get("sub_category_1"))
        sub_category_2 = re.sub(r'[^a-zA-Z0-9]', '', category.get("sub_category_2"))
        link = category.get("link")

        dir_name = f"./assets/product_links/{category_name}/"
        os.makedirs(dir_name, exist_ok=True)
        file_name = dir_name + f"{sub_category_1}_{sub_category_2}.json"

        products_list = parse_soup(scrape_webpage(link))

        with open(file_name, "w") as f:
            json.dump(products_list, f, indent=2)

        return file_name  # for progress

    except Exception as e:
        traceback.print_exc()
        print(f"Failed while scraping for category: {category.get('link')}")
        return None

In [13]:
subset = categories

with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(process_category, cat) for cat in subset]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping"):
        _ = future.result()

Scraping:   0%|          | 0/115 [00:00<?, ?it/s]Traceback (most recent call last):
  File "C:\Users\Rishi\AppData\Local\Temp\ipykernel_24288\3857703333.py", line 12, in process_category
    products_list = parse_soup(scrape_webpage(link))
                               ~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Rishi\AppData\Local\Temp\ipykernel_24288\1471710290.py", line 9, in scrape_webpage
    driver = webdriver.Chrome(options=chrome_options)
  File "d:\tortoise\cv\.venv\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 47, in __init__
    super().__init__(
    ~~~~~~~~~~~~~~~~^
        browser_name=DesiredCapabilities.CHROME["browserName"],
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
        keep_alive=keep_alive,
        ^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "d:\tortoise\cv\.venv\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 69, in __init__
    super().__init__(command_executor=executor, options=options)
   

Failed while scraping for category: https://www.bigbasket.com/pc/fruits-vegetables/fresh-vegetables/potato-onion-tomato/?nc=cs#!page=1


Scraping:   2%|▏         | 2/115 [00:45<48:14, 25.62s/it]

Total products in Cat:  19
Products with image:  10


Scraping:   3%|▎         | 3/115 [00:46<27:14, 14.59s/it]

...

Total products in Cat:  79
Products with image:  33


Scraping:  99%|█████████▉| 114/115 [24:30<00:09,  9.94s/it]

Total products in Cat:  14
Products with image:  14


Scraping: 100%|██████████| 115/115 [24:35<00:00, 12.83s/it]

Total products in Cat:  219
Products with image:  189


In [36]:
import os
import json
import re
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

OUTPUT_DIR = "./assets/images"

In [67]:
with open("./assets/categories.json", "r") as f:
    categories = json.load(f)

In [68]:
def sanitize_filename(text):
    return re.sub(r'[^a-zA-Z0-9]', '', text)[:50]

In [71]:
def download_image_and_metadata(args):
    category, product, category_meta = args

    image_link = product.get("image_link")
    if not image_link:
        return None

    name = product.get("name")
    ext = image_link.split(".")[-1].split("?")[0]
    filename = f"{sanitize_filename(name)}.{ext}"
    dirpath = os.path.join(OUTPUT_DIR, sanitize_filename(category_meta["category"]))
    os.makedirs(dirpath, exist_ok=True)
    filepath = os.path.join(dirpath, filename)

    try:
        if not os.path.exists(filepath):
            response = requests.get(image_link, timeout=10)
            if response.status_code == 200:
                with open(filepath, "wb") as f:
                    f.write(response.content)

        return {
            "category": category_meta["category"],
            "sub_category_1": category_meta["sub_category_1"],
            "sub_category_2": category_meta["sub_category_2"],
            "filename": os.path.abspath(filepath),
            "name": name,
            "sku_type": product.get("sku_type"),
            "rating": product.get("rating"),
            "image_url": image_link
        }

    except Exception as e:
        print(f"Error downloading {image_link}: {e}")
        return None

In [72]:
args_list = []

for category in categories:
    category_name = re.sub(r'[^a-zA-Z0-9]', '', category.get("category"))
    sub_category_1 = re.sub(r'[^a-zA-Z0-9]', '', category.get("sub_category_1"))
    sub_category_2 = re.sub(r'[^a-zA-Z0-9]', '', category.get("sub_category_2"))

    file_path = f"./assets/product_links/{category_name}/{sub_category_1}_{sub_category_2}.json"

    try:
        with open(file_path, 'r') as f:
            products = json.load(f)

        for product in products:
            args_list.append((category, product, {
                "category": category.get("category"),
                "sub_category_1": category.get("sub_category_1"),
                "sub_category_2": category.get("sub_category_2")
            }))
    except Exception as e:
        print(f"Could not load {file_path}: {e}")

Could not load ./assets/product_links/FruitsVegetables/FlowerBouquetsBunches_OtherFlowers.json: [Errno 2] No such file or directory: './assets/product_links/FruitsVegetables/FlowerBouquetsBunches_OtherFlowers.json'
Could not load ./assets/product_links/FruitsVegetables/FlowerBouquetsBunches_Roses.json: [Errno 2] No such file or directory: './assets/product_links/FruitsVegetables/FlowerBouquetsBunches_Roses.json'
Could not load ./assets/product_links/BakeryCakesDairy/Dairy_Cheese.json: [Errno 2] No such file or directory: './assets/product_links/BakeryCakesDairy/Dairy_Cheese.json'
Could not load ./assets/product_links/BakeryCakesDairy/BreadsBuns_BrownWheatMultigrain.json: [Errno 2] No such file or directory: './assets/product_links/BakeryCakesDairy/BreadsBuns_BrownWheatMultigrain.json'
Could not load ./assets/product_links/SnacksBrandedFoods/IndianMithai_ChikkiGajjak.json: [Errno 2] No such file or directory: './assets/product_links/SnacksBrandedFoods/IndianMithai_ChikkiGajjak.json'
Cou

In [73]:
metadata = []
with ThreadPoolExecutor(max_workers=16) as executor:
    for result in tqdm(executor.map(download_image_and_metadata, args_list), total=len(args_list), desc="Downloading"):
        if result:
            metadata.append(result)

Downloading: 100%|██████████| 32644/32644 [04:22<00:00, 124.54it/s]


In [74]:
metadata_df = pd.DataFrame(metadata)

metadata_df.to_csv("training_metadata.csv")

In [78]:
metadata_df["filename"][0]

'd:\\tortoise\\cv\\assets\\images\\FruitsVegetables\\Onion.jpg'